In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR='D:\Temp\MachineLearning\data\cifar-10-batches-py'

class   CifarData:
    def __init__(self,filelist,shuffle):
        all_data=[]
        all_labels=[]
        #循环读取多个批次的数据集文件
        for filename in filelist:
            data,labels=self.load_data(filename)      
            all_data.append(data)
            all_labels.append(labels)
        # 将所有data以纵向的方式合并到_data
        self._data=np.vstack(all_data)
        # 数据归一化处理
        self._data=self._data/127.5 - 1
        # 将所有label以横向方式合并到_label
        self._labels=np.hstack(all_labels)
        # 数据中样本个数
        self._example_num=self._data.shape[0]
        # print(self._example_num)
        # 定位数据集遍历的位置指针
        self._indicator=0
        # 根据传入的shuffle决定是否对数据进行打乱
        self._shuffle=shuffle
        if self._shuffle:
            self.shuffle_data()
                 
    def load_data(self,filename):
        """读取一个批文件中的数据"""
        with open(filename,'rb') as f:
            data=pickle.load(f,encoding='bytes')
            return data[b'data'],data[b'labels']
        
    def shuffle_data(self):
        # 返回一个打乱的序号列表，例如将0~5 随机打乱为 [5,3,2,4,0,1]
        shuffle_list=np.random.permutation(self._example_num)
        # 根据随机列表重排数据和对应的标签值
        self._data=self._data[shuffle_list]
        self._labels=self._labels[shuffle_list]
        
    def next_batch(self,batch_size ):
        '''返回当前指针self._indicator之后batch_size个数据'''
        end_indicator=self._indicator+batch_size
        # 如果相加结果超过了样本个数
        if end_indicator>self._example_num:
            # 如果可以洗牌，打乱数据并从0开始重新取数据
            if self._shuffle:
                self.shuffle_data()
                self._indicator=0
                end_indicator=batch_size
            else:
                raise Exception("没有更多数据了！")    
        # 返回从_indicator到end_indicator的data和labels，并将指针后移到end_indicator
        batch_data=self._data[self._indicator:end_indicator]
        batch_labels=self._labels[self._indicator:end_indicator]
        self._indicator=end_indicator
        return batch_data,batch_labels

# 拼接训练数据的路径名        
train_file=[os.path.join(CIFAR_DIR,'data_batch_%d'%i) for i in range (1,6)]
test_file=[os.path.join(CIFAR_DIR,'test_batch')]
# 创建训练数据对象
train_data=CifarData(train_file,True)     

def res_block(x,out_chanel):
    '''实现残差连接块'''
    in_chanel=x.shape[-1]
    # 如果输出通道是输入的两倍，说明需要进行降采样操作
    if in_chanel*2==out_chanel:
        increase_dim=True
        strides=(2,2)           # 将步长设为2，卷积操作会进行降采样
    elif in_chanel==out_chanel:
        increase_dim=False
        strides=(1,1)           # 步长为1，不进行降采样
    else:
        raise Exception('输入输出通道数目无法匹配')
    # 对输入进行卷积操作
    conv1=tf.layers.conv2d(x,out_chanel,(3,3),strides,padding='same',activation=tf.nn.relu,name='conv1')
    conv2=tf.layers.conv2d(conv1,out_chanel,(3,3),(1,1),padding='same',activation=tf.nn.relu,name='conv2')
    # 如果需要降采样，则对恒等变换部分进行降采样，并补充通道数
    if increase_dim:
        pool_x=tf.layers.average_pooling2d(x,(2,2),(2,2))
        # pool_x为[None,img_width,img_height,chanel]，通过tf.pad()对第四维的左右各补充半个in_chanel，使其翻倍
        pad_x=tf.pad(pool_x,[[0,0],[0,0],[0,0],[in_chanel//2,in_chanel//2]])
    else:
        pad_x=x         # 不需要降采样，残差直接为x
    out_x=conv2+pad_x   # 将卷积结果和残差相加得到输出
    return out_x

def res_net(x,block_num_list,filter_base,class_num):
    '''构建残差网络
    -x,输入
    -block_num_list,残差连接块数列表，例如[3,4,6,3]
    -filter_base,初始通道数
    -class_num,Cifar数据集的类别数，即最后网络输出的通道数
    '''
    layer_num=len(block_num_list)
    layers=[] 
    # 截取x的后三个维度信息，[None,width,height,chanel]->[width,height,chanel]
    input_size=x.shape[1:]
    with tf.variable_scope('conv0'):
        # 首先进行一次卷积操作，并将结果放到layers列表
        conv0=tf.layers.conv2d(x,filter_base,(3,3),(1,1),padding='same',activation=tf.nn.relu,name='conv0')
        layers.append(conv0)
    # 根据残差连接块数列表，循环实现多个残差连接块
    for layer_id in range(layer_num):
        for i in range(block_num_list[layer_id]):
            with tf.variable_scope('conv%d_%d'%(layer_id,i)):
                # 输出通道数是在初始通道的基础上，每经过一层翻一倍，乘以2的layer_id次方
                out_chanel=filter_base*(2**layer_id)
                conv=res_block(layers[-1],out_chanel)
                layers.append(conv)
    # 通过断言检查输出维度是否符合预期，如不符合，报错停止
    multiplier=2 ** (layer_num - 1)    
    assert layers[-1].get_shape().as_list()[1:] \
        == [input_size[0] // multiplier,
            input_size[1] // multiplier,
            filter_base * multiplier]
    # 最后经过一个全连接层将结果输出
    with tf.variable_scope('fc'):
        # layers[-1]:[None,width,height,chanel]，在width，height上求平均
        global_pool=tf.reduce_mean(layers[-1],[1,2])
        logits=tf.layers.dense(global_pool,class_num)
        layers.append(logits)
    return layers[-1]

x=tf.placeholder(tf.float32,[None,3072])
y=tf.placeholder(tf.int64,[None])
# 将一维向量x转化为具有三通道的多维图片，并交换向量通道
x_img=tf.reshape(x,[-1,3,32,32])
x_img=tf.transpose(x_img,[0,2,3,1])

y_predict=res_net(x_img,[3,4,6,3],32,10)
# 使用交叉熵作为损失函数
loss=tf.losses.sparse_softmax_cross_entropy(y,y_predict)

# 将标签预测向量中最大的下标作为预测值，例如[0.1,0.8...0.01]，则预测为第二类
predict=tf.math.argmax(y_predict,1)
# 通过equal函数逐一比较predict，y_reshape的每一个元素
correction=tf.equal(predict,y)
accuracy=tf.reduce_mean(tf.cast(correction,tf.float64))
#定义优化方法
with tf.name_scope('train_op'):
    train_op=tf.train.AdamOptimizer(1e-3).minimize(loss)
    
init=tf.global_variables_initializer()
batch_size=20
train_steps=5000
test_steps=100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data,batch_labels=train_data.next_batch(batch_size)
        loss_val,acc_val,_=sess.run([loss,accuracy,train_op],feed_dict={x:batch_data,y:batch_labels})
        if (i+1) % 500==0:
            print("第%d步：损失：%.5f，精确度：%.5f"%(i,loss_val,acc_val))
        #每训练1000次，在test数据集上进行一次测试
        if (i+1)%1000==0:
            # 定义测试集数据对象
            test_data=CifarData(test_file,False)   
            all_acc=[]
            for j in range(test_steps):
                test_batch_data,test_batch_labels=test_data.next_batch(batch_size)
                test_acc=sess.run([accuracy],feed_dict={x:test_batch_data,y:test_batch_labels})
                all_acc.append(test_acc)
            # 将测得的多个准确率求均值
            test_acc_mean=np.mean(all_acc)
            print("第%d步测试集准确率%.4f"%(i,test_acc_mean))
            
            

D:\Program\IDE\Anaconda\envs\TF-GPU\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Program\IDE\Anaconda\envs\TF-GPU\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Program\IDE\Anaconda\envs\TF-GPU\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Program\IDE\Anaconda\envs\TF-GPU\lib\site-packages\tensorflow\python\f

50000
第499步：损失：1.80951，精确度：0.30000
第999步：损失：1.53882，精确度：0.50000
10000
第999步测试集准确率0.4415
第1499步：损失：1.45390，精确度：0.45000
第1999步：损失：1.43906，精确度：0.55000
10000
第1999步测试集准确率0.5595
第2499步：损失：0.96445，精确度：0.70000
第2999步：损失：1.00230，精确度：0.70000
10000
第2999步测试集准确率0.6270
第3499步：损失：0.95330，精确度：0.60000
第3999步：损失：0.41390，精确度：1.00000
10000
第3999步测试集准确率0.6465
第4499步：损失：0.70792，精确度：0.80000
第4999步：损失：0.88312，精确度：0.65000
10000
第4999步测试集准确率0.6765
